In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

D:\anaconda\envs\torch-1.13.1-py-3.8\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import torch
import torch.nn as nn
x_input=torch.randn(3,3)#随机生成输入 
print('x_input:\n',x_input) 
y_target=torch.tensor([1,2,0])#设置输出具体值 print('y_target\n',y_target)

#计算输入softmax，此时可以看到每一行加到一起结果都是1
softmax_func=nn.Softmax(dim=1)
soft_output=softmax_func(x_input)
print('soft_output:\n',soft_output)
 
#对比softmax与log的结合与nn.LogSoftmaxloss(负对数似然损失)的输出结果，发现两者是一致的。
logsoftmax_func=nn.LogSoftmax(dim=1)
logsoftmax_output=logsoftmax_func(x_input)
print('logsoftmax_output:\n',logsoftmax_output)

#pytorch中关于NLLLoss的默认参数配置为：reducetion=True、size_average=True ,根据标签把对应的结果取出来求平均再去掉负号
nllloss_func=nn.NLLLoss()
nlloss_output=nllloss_func(logsoftmax_output,y_target)
print('nlloss_output:\n',nlloss_output)

#直接使用pytorch中的loss_func=nn.CrossEntropyLoss()看与经过NLLLoss的计算是不是一样
crossentropyloss=nn.CrossEntropyLoss()
crossentropyloss_output=crossentropyloss(x_input, y_target)
print('crossentropyloss_output:\n',crossentropyloss_output)

x_input:
 tensor([[ 0.0054, -1.0517, -0.4395],
        [-0.9735, -0.4136, -0.4568],
        [-0.0906, -1.4874,  0.7973]])
soft_output:
 tensor([[0.5029, 0.1748, 0.3223],
        [0.2259, 0.3954, 0.3787],
        [0.2719, 0.0673, 0.6608]])
logsoftmax_output:
 tensor([[-0.6873, -1.7444, -1.1322],
        [-1.4877, -0.9278, -0.9710],
        [-1.3022, -2.6990, -0.4143]])
nlloss_output:
 tensor(1.3392)
crossentropyloss_output:
 tensor(1.3392)


In [3]:
training_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [33]:
for i in training_data:
    print(i[0].shape, i[1])
    break

for x, y in train_dataloader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([1, 28, 28]) 9
torch.Size([64, 1, 28, 28])
torch.Size([64])


In [35]:
learning_rate = 1e-3
batch_size = 64
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100 == 0:
            loss, current = loss.item(), batch*batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
epochs = 3
for t  in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.792460  [   64/60000]
loss: 0.870733  [ 6464/60000]
loss: 0.632614  [12864/60000]
loss: 0.848078  [19264/60000]
loss: 0.742331  [25664/60000]
loss: 0.728443  [32064/60000]
loss: 0.818450  [38464/60000]
loss: 0.776703  [44864/60000]
loss: 0.800395  [51264/60000]
loss: 0.760595  [57664/60000]
Accuracy: 72.1%, Avg loss: 0.760791 

Epoch 2
-------------------------------
loss: 0.753925  [   64/60000]
loss: 0.840109  [ 6464/60000]
loss: 0.600129  [12864/60000]
loss: 0.823758  [19264/60000]
loss: 0.720855  [25664/60000]
loss: 0.702934  [32064/60000]
loss: 0.793813  [38464/60000]
loss: 0.759571  [44864/60000]
loss: 0.778112  [51264/60000]
loss: 0.739770  [57664/60000]
Accuracy: 73.2%, Avg loss: 0.738399 

Epoch 3
-------------------------------
loss: 0.720256  [   64/60000]
loss: 0.812392  [ 6464/60000]
loss: 0.572530  [12864/60000]
loss: 0.803252  [19264/60000]
loss: 0.702219  [25664/60000]
loss: 0.681940  [32064/60000]
loss: 0.771314  [38464/6

In [37]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[ 5.5497e-03, -1.0505e-02, -1.7027e-02,  ..., -8.1669e-03,
                        3.5819e-02,  2.1796e-02],
                      [ 1.5335e-02, -2.9324e-02,  1.6550e-02,  ..., -1.8013e-02,
                        1.4317e-02, -5.4089e-05],
                      [ 5.6648e-03, -2.6010e-02, -2.9144e-03,  ...,  7.4799e-03,
                        1.1316e-03, -2.8733e-02],
                      ...,
                      [-2.2052e-02,  7.9496e-04,  1.4152e-02,  ..., -4.1585e-04,
                        2.7763e-02,  1.0144e-02],
                      [ 7.9229e-03, -2.6692e-02,  2.0166e-02,  ..., -1.6596e-02,
                       -8.1415e-03,  3.1815e-02],
                      [-3.3667e-02, -1.4509e-02, -1.9302e-02,  ...,  1.9705e-02,
                       -1.1895e-02, -2.8388e-02]])),
             ('linear_relu_stack.0.bias',
              tensor([-1.3159e-02, -6.8923e-03, -4.9877e-03, -2.7703e-02, -8.1573e-03,
           